In [1]:
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"
from nibabel.testing import data_path
import nibabel as nib
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import time
import datetime
import json

In [2]:
import itertools
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from sklearn import preprocessing

In [3]:
#search_dir = "/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/gradients/train_totdice/"
search_dir ="/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train_SGD/gradients/train_dice1"
os.chdir(search_dir)
file_list_gradtrain = filter(os.path.isfile, os.listdir(search_dir))
file_list_gradtrain = [os.path.join(search_dir, f) for f in file_list_gradtrain] # add path to each file
file_list_gradtrain.sort(key=lambda x: os.path.getmtime(x))

In [4]:
search_dir = "/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train3/gradients/test_dice2/"
#search_dir ="/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train1/gradients/test_totdice/"
os.chdir(search_dir)
file_list_gradtest = filter(os.path.isfile, os.listdir(search_dir))
file_list_gradtest = [os.path.join(search_dir, f) for f in file_list_gradtest] # add path to each file
file_list_gradtest.sort(key=lambda x: os.path.getmtime(x))

FileNotFoundError: [Errno 2] No such file or directory: '/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train3/gradients/test_dice2/'

In [5]:
search_dir = "/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_train/"
os.chdir(search_dir)
file_list_train = filter(os.path.isfile, os.listdir(search_dir))
file_list_train = [os.path.join(search_dir, f) for f in file_list_train] # add path to each file
file_list_train.sort(key=lambda x: os.path.getmtime(x))

In [6]:
search_dir = "/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Data/DataTracin_test/"
os.chdir(search_dir)
file_list_test = filter(os.path.isfile, os.listdir(search_dir))
file_list_test = [os.path.join(search_dir, f) for f in file_list_test] # add path to each file
file_list_test.sort(key=lambda x: os.path.getmtime(x))

In [7]:
file = glob(os.path.join("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train/gradients/train_totdice/", "*"))

In [8]:
def load_grad_train(file):
    data = np.load(file, allow_pickle=True)
    file = str(file)
    index = "/".join(file.split("/", file.count("/"))[file.count("/"):])
    index = int(index[10:len(index)-5])
    return index, data['grad_test']

def load_grad_test(file):
    data = np.load(file, allow_pickle=True)
    file = str(file)
    index = "/".join(file.split("/", file.count("/"))[file.count("/"):])
    index = int(index[9:len(index)-5])
    return index, data['grad_test']

In [9]:
def load_image_train(image_file):
    data = np.load(image_file)
    image_file = str(image_file)
    index = "/".join(image_file.split("/", image_file.count("/"))[image_file.count("/"):])
    index = int(index[6:len(index)-5])
    return index, data['X_train'], data['Y_train']

def load_image_test(image_file):
    data = np.load(image_file)
    image_file = str(image_file)
    index = "/".join(image_file.split("/", image_file.count("/"))[image_file.count("/"):])
    index = int(index[5:len(index)-5])
    return index, data['X_test'], data['Y_test']

def load_image_val(image_file):
    data = np.load(image_file)
    image_file = str(image_file)
    index = "/".join(image_file.split("/", image_file.count("/"))[image_file.count("/"):])
    index = int(index[4:len(index)-5])
    return index, data['X_val'], data['Y_val']


def test_reshape(index, X,Y):
    X = tf.cast(X, tf.float32)
    Y = tf.cast(Y, tf.float32)
    X = tf.reshape(X, [192,192,4])
    Y = tf.reshape(Y, [192,192,4])
    return index,X,Y

BATCH_SIZE = 1

In [10]:
train_grad = tf.data.Dataset.list_files(file_list_gradtrain, shuffle = False)
train_grad = train_grad.map(lambda item: tf.numpy_function(
          load_grad_train, [item], [tf.int64, tf.float32]),
          num_parallel_calls=tf.data.AUTOTUNE)

2023-10-09 08:16:38.499901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30967 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:85:00.0, compute capability: 7.0


In [8]:
test_grad = tf.data.Dataset.list_files(file_list_gradtest, shuffle = False)
test_grad = test_grad.map(lambda item: tf.numpy_function(
          load_grad_test, [item], [tf.int64, tf.float32]),
          num_parallel_calls=tf.data.AUTOTUNE)

NameError: name 'file_list_gradtest' is not defined

In [11]:
train_dataset = tf.data.Dataset.list_files(file_list_train, shuffle = False)
train_dataset = train_dataset.map(lambda item: tf.numpy_function(
          load_image_train, [item], [tf.int64, tf.double, tf.double]),
          num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(test_reshape)
train_dataset = train_dataset.batch(BATCH_SIZE)

In [12]:
test_dataset = tf.data.Dataset.list_files(file_list_test, shuffle = False)
test_dataset = test_dataset.map(lambda item: tf.numpy_function(
          load_image_test, [item], [tf.int64, tf.double, tf.double]),
          num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(test_reshape)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [13]:
map_test = {}
map_train = {}
for i,j in enumerate(train_dataset):
    map_train[int(j[0])] = i

2023-10-09 08:16:44.769576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [ ]:
for i,j in enumerate(test_dataset):
    map_test[int(j[0])] = i
    

In [14]:
def tracin(grad1,grad2):
    s = grad1@grad2.transpose()
    s = np.array(s, dtype = float)
    
    return s

In [15]:
def my_gen(train_grad, test_grad):
    
    score = {}
    
    for i,j in enumerate(train_grad):
        score[int(j[0])] = np.trace(tracin(np.array(test_grad), np.array(j[1])))
        if i%100 == 1 : print("x", end =" " )
    
    return score
    

In [16]:
final_score = {}
for i,j in enumerate(train_grad):
    final_score[int(j[0])] = my_gen(train_grad, j[1])
    print(f"test_{j[0]}")

x x x x x x x x x x x x x x x x x x x x x test_0
x x x x x x x x x x x x x x x x x x x x x test_1
x x x x x x x x x x x x x x x x x x x x x test_10
x x x x x x x x x x x x x x x x x x x x x test_100
x x x x x x x x x x x x x x x x x x x x x test_1000
x x x x x x x x x x x x x x x x x x x x x test_1001
x x x x x x x x x x x x x x x x x x x x x test_1002
x x x x x x x x x x x x x x x x x x x x x test_1003
x x x x x x x x x x x x x x x x x x x x x test_1004
x x x x x x x x x x x x x x x x x x x x x test_1005
x x x x x x x x x x x x x x x x x x x x x test_1006
x x x x x x x x x x x x x x x x x x x x x test_1007
x x x x x x x x x x x x x x x x x x x x x test_1008
x x x x x x x x x x x x x x x x x x x x x test_1009
x x x x x x x x x x x x x x x x x x x x x test_101
x x x x x x x x x x x x x x x x x x x x x test_1010
x x x x x x x x x x x x x x x x x x x x x test_1011
x x x x x x x x x x x x x x x x x x x x x test_1012
x x x x x x x x x x x x x x x x x x x x x test_1013
x x x x x x x x x x x 

x x x x x x x x x x x x x x x x x x x x x test_114
x x x x x x x x x x x x x x x x x x x x x test_1140
x x x x x x x x x x x x x x x x x x x x x test_1141
x x x x x x x x x x x x x x x x x x x x x test_1142
x x x x x x x x x x x x x x x x x x x x x test_1143
x x x x x x x x x x x x x x x x x x x x x test_1144
x x x x x x x x x x x x x x x x x x x x x test_1145
x x x x x x x x x x x x x x x x x x x x x test_1146
x x x x x x x x x x x x x x x x x x x x x test_1147
x x x x x x x x x x x x x x x x x x x x x test_1148
x x x x x x x x x x x x x x x x x x x x x test_1149
x x x x x x x x x x x x x x x x x x x x x test_115
x x x x x x x x x x x x x x x x x x x x x test_1150
x x x x x x x x x x x x x x x x x x x x x test_1151
x x x x x x x x x x x x x x x x x x x x x test_1152
x x x x x x x x x x x x x x x x x x x x x test_1153
x x x x x x x x x x x x x x x x x x x x x test_1154
x x x x x x x x x x x x x x x x x x x x x test_1155
x x x x x x x x x x x x x x x x x x x x x test_1156
x x x x x x x 

x x x x x x x x x x x x x x x x x x x x x test_1282
x x x x x x x x x x x x x x x x x x x x x test_1283
x x x x x x x x x x x x x x x x x x x x x test_1284
x x x x x x x x x x x x x x x x x x x x x test_1285
x x x x x x x x x x x x x x x x x x x x x test_1286
x x x x x x x x x x x x x x x x x x x x x test_1287
x x x x x x x x x x x x x x x x x x x x x test_1288
x x x x x x x x x x x x x x x x x x x x x test_1289
x x x x x x x x x x x x x x x x x x x x x test_129
x x x x x x x x x x x x x x x x x x x x x test_1290
x x x x x x x x x x x x x x x x x x x x x test_1291
x x x x x x x x x x x x x x x x x x x x x test_1292
x x x x x x x x x x x x x x x x x x x x x test_1293
x x x x x x x x x x x x x x x x x x x x x test_1294
x x x x x x x x x x x x x x x x x x x x x test_1295
x x x x x x x x x x x x x x x x x x x x x test_1296
x x x x x x x x x x x x x x x x x x x x x test_1297
x x x x x x x x x x x x x x x x x x x x x test_1298
x x x x x x x x x x x x x x x x x x x x x test_1299
x x x x x x x

x x x x x x x x x x x x x x x x x x x x x test_1424
x x x x x x x x x x x x x x x x x x x x x test_1425
x x x x x x x x x x x x x x x x x x x x x test_1426
x x x x x x x x x x x x x x x x x x x x x test_1427
x x x x x x x x x x x x x x x x x x x x x test_1428
x x x x x x x x x x x x x x x x x x x x x test_1429
x x x x x x x x x x x x x x x x x x x x x test_143
x x x x x x x x x x x x x x x x x x x x x test_1430
x x x x x x x x x x x x x x x x x x x x x test_1431
x x x x x x x x x x x x x x x x x x x x x test_1432
x x x x x x x x x x x x x x x x x x x x x test_1433
x x x x x x x x x x x x x x x x x x x x x test_1434
x x x x x x x x x x x x x x x x x x x x x test_1435
x x x x x x x x x x x x x x x x x x x x x test_1436
x x x x x x x x x x x x x x x x x x x x x test_1437
x x x x x x x x x x x x x x x x x x x x x test_1438
x x x x x x x x x x x x x x x x x x x x x test_1439
x x x x x x x x x x x x x x x x x x x x x test_144
x x x x x x x x x x x x x x x x x x x x x test_1440
x x x x x x x 

x x x x x x x x x x x x x x x x x x x x x test_1567
x x x x x x x x x x x x x x x x x x x x x test_1568
x x x x x x x x x x x x x x x x x x x x x test_1569
x x x x x x x x x x x x x x x x x x x x x test_157
x x x x x x x x x x x x x x x x x x x x x test_1570
x x x x x x x x x x x x x x x x x x x x x test_1571
x x x x x x x x x x x x x x x x x x x x x test_1572
x x x x x x x x x x x x x x x x x x x x x test_1573
x x x x x x x x x x x x x x x x x x x x x test_1574
x x x x x x x x x x x x x x x x x x x x x test_1575
x x x x x x x x x x x x x x x x x x x x x test_1576
x x x x x x x x x x x x x x x x x x x x x test_1577
x x x x x x x x x x x x x x x x x x x x x test_1578
x x x x x x x x x x x x x x x x x x x x x test_1579
x x x x x x x x x x x x x x x x x x x x x test_158
x x x x x x x x x x x x x x x x x x x x x test_1580
x x x x x x x x x x x x x x x x x x x x x test_1581
x x x x x x x x x x x x x x x x x x x x x test_1582
x x x x x x x x x x x x x x x x x x x x x test_1583
x x x x x x x 

...

x x x x x x x x x x x x x x x x x x x x x test_274
x x x x x x x x x x x x x x x x x x x x x test_275
x x x x x x x x x x x x x x x x x x x x x test_276
x x x x x x x x x x x x x x x x x x x x x test_277
x x x x x x x x x x x x x x x x x x x x x test_278
x x x x x x x x x x x x x x x x x x x x x test_279
x x x x x x x x x x x x x x x x x x x x x test_28
x x x x x x x x x x x x x x x x x x x x x test_280
x x x x x x x x x x x x x x x x x x x x x test_281
x x x x x x x x x x x x x x x x x x x x x test_282
x x x x x x x x x x x x x x x x x x x x x test_283
x x x x x x x x x x x x x x x x x x x x x test_284
x x x x x x x x x x x x x x x x x x x x x test_285
x x x x x x x x x x x x x x x x x x x x x test_286
x x x x x x x x x x x x x x x x x x x x x test_287
x x x x x x x x x x x x x x x x x x x x x test_288
x x x x x x x x x x x x x x x x x x x x x test_289
x x x x x x x x x x x x x x x x x x x x x test_29
x x x x x x x x x x x x x x x x x x x x x test_290
x x x x x x x x x x x x x x x x x

x x x x x x x x x x x x x x x x x x x x x test_419
x x x x x x x x x x x x x x x x x x x x x test_42
x x x x x x x x x x x x x x x x x x x x x test_420
x x x x x x x x x x x x x x x x x x x x x test_421
x x x x x x x x x x x x x x x x x x x x x test_422
x x x x x x x x x x x x x x x x x x x x x test_423
x x x x x x x x x x x x x x x x x x x x x test_424
x x x x x x x x x x x x x x x x x x x x x test_425
x x x x x x x x x x x x x x x x x x x x x test_426
x x x x x x x x x x x x x x x x x x x x x test_427
x x x x x x x x x x x x x x x x x x x x x test_428
x x x x x x x x x x x x x x x x x x x x x test_429
x x x x x x x x x x x x x x x x x x x x x test_43
x x x x x x x x x x x x x x x x x x x x x test_430
x x x x x x x x x x x x x x x x x x x x x test_431
x x x x x x x x x x x x x x x x x x x x x test_432
x x x x x x x x x x x x x x x x x x x x x test_433
x x x x x x x x x x x x x x x x x x x x x test_434
x x x x x x x x x x x x x x x x x x x x x test_435
x x x x x x x x x x x x x x x x x

x x x x x x x x x x x x x x x x x x x x x test_564
x x x x x x x x x x x x x x x x x x x x x test_565
x x x x x x x x x x x x x x x x x x x x x test_566
x x x x x x x x x x x x x x x x x x x x x test_567
x x x x x x x x x x x x x x x x x x x x x test_568
x x x x x x x x x x x x x x x x x x x x x test_569
x x x x x x x x x x x x x x x x x x x x x test_57
x x x x x x x x x x x x x x x x x x x x x test_570
x x x x x x x x x x x x x x x x x x x x x test_571
x x x x x x x x x x x x x x x x x x x x x test_572
x x x x x x x x x x x x x x x x x x x x x test_573
x x x x x x x x x x x x x x x x x x x x x test_574
x x x x x x x x x x x x x x x x x x x x x test_575
x x x x x x x x x x x x x x x x x x x x x test_576
x x x x x x x x x x x x x x x x x x x x x test_577
x x x x x x x x x x x x x x x x x x x x x test_578
x x x x x x x x x x x x x x x x x x x x x test_579
x x x x x x x x x x x x x x x x x x x x x test_58
x x x x x x x x x x x x x x x x x x x x x test_580
x x x x x x x x x x x x x x x x x

x x x x x x x x x x x x x x x x x x x x x test_709
x x x x x x x x x x x x x x x x x x x x x test_71
x x x x x x x x x x x x x x x x x x x x x test_710
x x x x x x x x x x x x x x x x x x x x x test_711
x x x x x x x x x x x x x x x x x x x x x test_712
x x x x x x x x x x x x x x x x x x x x x test_713
x x x x x x x x x x x x x x x x x x x x x test_714
x x x x x x x x x x x x x x x x x x x x x test_715
x x x x x x x x x x x x x x x x x x x x x test_716
x x x x x x x x x x x x x x x x x x x x x test_717
x x x x x x x x x x x x x x x x x x x x x test_718
x x x x x x x x x x x x x x x x x x x x x test_719
x x x x x x x x x x x x x x x x x x x x x test_72
x x x x x x x x x x x x x x x x x x x x x test_720
x x x x x x x x x x x x x x x x x x x x x test_721
x x x x x x x x x x x x x x x x x x x x x test_722
x x x x x x x x x x x x x x x x x x x x x test_723
x x x x x x x x x x x x x x x x x x x x x test_724
x x x x x x x x x x x x x x x x x x x x x test_725
x x x x x x x x x x x x x x x x x

x x x x x x x x x x x x x x x x x x x x x test_854
x x x x x x x x x x x x x x x x x x x x x test_855
x x x x x x x x x x x x x x x x x x x x x test_856
x x x x x x x x x x x x x x x x x x x x x test_857
x x x x x x x x x x x x x x x x x x x x x test_858
x x x x x x x x x x x x x x x x x x x x x test_859
x x x x x x x x x x x x x x x x x x x x x test_86
x x x x x x x x x x x x x x x x x x x x x test_860
x x x x x x x x x x x x x x x x x x x x x test_861
x x x x x x x x x x x x x x x x x x x x x test_862
x x x x x x x x x x x x x x x x x x x x x test_863
x x x x x x x x x x x x x x x x x x x x x test_864
x x x x x x x x x x x x x x x x x x x x x test_865
x x x x x x x x x x x x x x x x x x x x x test_866
x x x x x x x x x x x x x x x x x x x x x test_867
x x x x x x x x x x x x x x x x x x x x x test_868
x x x x x x x x x x x x x x x x x x x x x test_869
x x x x x x x x x x x x x x x x x x x x x test_87
x x x x x x x x x x x x x x x x x x x x x test_870
x x x x x x x x x x x x x x x x x

In [17]:
np.save("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Crossvalidation/Train_SGD/train_vs_train_tracin_score_dice1.npy", final_score) 

#final_score = np.load("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Radiomic_Features/final_score_totdice.npy",allow_pickle='TRUE').item()

In [ ]:
b = {}

for k in range(len(final_score)):
    print(f"test_{k}")
    c = {}
    for j in range(0, len(final_score[k]), 10):
        m = 0 
        a = 0
        
        for i in range(j,j+10):
            a += final_score[k][i]
        
            if abs(final_score[k][i]) > m:
                m = abs(final_score[k][i])
                index = i
                
        print(f"max : {m}, index :{index}")
        c[index] = a
    print("\t")
    b[k] = c 

In [ ]:
#np.save("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Radiomic_Features/score_totdice.npy", b) 

b = np.load("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Radiomic_Features/score_totdice.npy",allow_pickle='TRUE').item()

In [ ]:
proponents_score = [sorted(b[i].items(), key=lambda x: x[1], reverse = True) for i in list(final_score.keys())[0:3]]

In [ ]:
proponents_score = [sorted(b[i].items(), key=lambda x: x[1], reverse = True) for i in range(len(b))]
opponents_score = [sorted(b[i].items(), key=lambda x: x[1]) for i in range(len(b))]

In [ ]:
np.savez(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Radiomic_Features/prop_index_dice1.npz", proponents = proponents_index)

In [ ]:
proponents_index = np.zeros((len(proponents_score), 207))
for i,j in enumerate(proponents_score):
    for k,h in enumerate(proponents_score[i]):
        proponents_index[i][k] = int(h[0])

In [ ]:
opponents_index = np.zeros((len(opponents_score), 207))
for i,j in enumerate(opponents_score):
    for k,h in enumerate(opponents_score[i]):
        opponents_index[i][k] = int(h[0])

In [ ]:
l = []
for j,i in enumerate(proponents_index):
    if i[0]+i[1]+i[2]+i[3] == 60:
        l.append(j)
prop = np.delete(proponents_index, l, axis = 0)

In [ ]:
m = {}
l = np.unique(prop).astype("int")
for j in l:
    count = 0
    for i in range(prop.shape[0]):
        if np.where(prop[i] == j)[0]<=30 or np.where(prop[i] == j)[0]>=len(prop)-30:
            count += 1
    m[j] = count
        

In [ ]:
new_m = {}
for i in sorted(m.items(), key=lambda x: x[1], reverse = True):
    if i[1]/len(prop)<= 0.3:
        new_m[i[0]] = i[1] 

In [ ]:
courses = range(len(list(new_m.keys())))
values = [i/len(prop) for i in list(new_m.values())]
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(courses, values, color ='maroon',
        width = 3)

In [ ]:
p = np.zeros(len(proponents_index), dtype = "object")
for k,i in enumerate(proponents_index):
    p[k] = [int(j) for j in i]

In [ ]:
p = np.zeros(len(prop), dtype = "object")
for k,i in enumerate(prop):
    p[k] = [int(j) for j in i if j in list(new_m.keys())]

In [ ]:
for i in range(0, len(proponents_index), 10):
    print(f"index : {int(i)}, {proponents_index[int(i)]}")
    if i == 210: break

In [ ]:
for i in range(0, len(opponents_index), 10):
    print(f"index : {int(i)}, {opponents_index[int(i)]}")
    if i == 210: break

In [ ]:
def dice0(y_true, y_pred, smooth = 1e-7):
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,0], 'float32'), [len(y_true), -1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,0], 'float32'), [len(y_true), -1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f), axis = 1))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2, axis = 1)+smooth)
def dice1(y_true, y_pred, smooth = 1e-7):  
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,1], 'float32'), [len(y_true), -1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,1], 'float32'), [len(y_true), -1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f), axis = 1))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2, axis = 1)+smooth)

def dice2(y_true, y_pred, smooth = 1e-7):
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,2], 'float32'), [len(y_true), -1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,2], 'float32'), [len(y_true), -1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f), axis = 1))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2, axis = 1)+smooth)

def dice3(y_true, y_pred, smooth = 1e-7):  
    y_true_f = tf.reshape(tf.cast(y_true[:,:,:,3], 'float32'), [len(y_true), -1]) 
    y_pred_f = tf.reshape(tf.cast(y_pred[:,:,:,3], 'float32'), [len(y_true), -1])
    return (2*tf.reduce_sum(tf.abs(y_true_f*y_pred_f), axis = 1))/(tf.reduce_sum(
        y_true_f**2 + y_pred_f**2, axis = 1)+smooth)

def dice_loss1(y_true, y_pred):
    a0 = 0
    a1 = 1
    a2 = 1
    a3 = 1
    return 1-(a0*dice0(y_true,y_pred)+a1*dice1(y_true,y_pred)+a2*dice2(
        y_true,y_pred)+a3*dice3(y_true,y_pred))/(a0+a1+a2+a3)

In [ ]:
checkpoint_dir = '/home/tordatom/Dati_Imaging/BraTs_19/checkpoints/training_checkpoints_Unet'
file_list_ckpt = glob(os.path.join(checkpoint_dir, "*"))
file_list_ckpt.sort()

In [ ]:
model = tf.keras.models.load_model(file_list_ckpt[-1], custom_objects={'dice0': dice0, 
                                                          'dice1': dice1,
                                                          'dice2': dice2, 
                                                          'dice3': dice3, 
                                                          "dice_loss1":dice_loss1})

In [ ]:
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images")

In [ ]:
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/totdice")

In [ ]:
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/test")
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/dice1")
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/dice2")
os.mkdir("/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/dice3")

In [ ]:
test = [0,1,10]

In [ ]:
test = list(final_score.keys())[0:3]

In [ ]:
test = np.arange(0,570,10)

In [ ]:
for i in test:
    print(f"------------test_{i}------------")
    try:
        os.mkdir(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/new_totdice/proponents_of_test_{i}")
        os.mkdir(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/new_totdice/opponents_of_test_{i}")
    except: 
        pass
    channel = next(itertools.islice(test_dataset, map_test[i], map_test[i]+1))[1]
    seg = next(itertools.islice(test_dataset, map_test[i], map_test[i]+1))[2]
    plt.figure(figsize=(12, 8))
    plt.subplot(131)
    plt.title(f"Channel 3, index: {i}")
    plt.imshow(channel[0,:,:,3])
    plt.subplot(132)
    plt.title("Segmentation")
    s = np.argmax(seg[0], axis = 2)
    #s[s!=1] = 0
    plt.imshow(s)
    plt.subplot(133)
    plt.title("Pred Segmentation")
    s_pred = np.argmax(model.predict(channel)[0], axis = 2)
    #s_pred[s_pred!=1] = 0
    plt.imshow(s_pred)
    plt.savefig(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/new_test/test_example_{i}.png")
    plt.show()
    for j in range(3):
        index = int(p[map_test[i]][j])
        channel = next(itertools.islice(train_dataset, map_train[index],
                                        map_train[index]+1))[1]
        seg = next(itertools.islice(train_dataset, map_train[index],
                                    map_train[index]+1))[2]
        plt.figure(figsize=(12, 8))
        print(f"------------proponent_{j}_for_test_{i}------------")
        plt.subplot(131)
        plt.title(f"Channel 3, index: {index}")
        plt.imshow(channel[0,:,:,3])
        plt.subplot(132)
        plt.title("Segmentation")
        s = np.argmax(seg[0], axis = 2)
        #s[s!=1] = 0
        plt.imshow(s)
        plt.subplot(133)
        plt.title("Pred Segmentation")
        s_pred = np.argmax(model.predict(channel)[0], axis = 2)
        #s_pred[s_pred!=1] = 0
        plt.imshow(s_pred)
        plt.savefig(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/new_totdice/proponents_of_test_{i}/train_proponent_{index}.png")
        plt.show()
        
        index = int(p[map_test[i]][len(p[map_test[i]])-j-1])
        channel = next(itertools.islice(train_dataset, map_train[index],
                                        map_train[index]+1))[1]
        seg = next(itertools.islice(train_dataset, map_train[index],
                                    map_train[index]+1))[2]

        plt.figure(figsize=(12, 8))
        print(f"------------opponent_{j}_for_test_{i}------------")
        plt.subplot(131)
        plt.title(f"Channel 3, index: {index}")
        plt.imshow(channel[0,:,:,3])
        plt.subplot(132)
        plt.title("Segmentation")
        s = np.argmax(seg[0], axis = 2)
        #s[s!=1] = 0
        plt.imshow(s)
        plt.subplot(133)
        plt.title("Pred Segmentation")
        s_pred = np.argmax(model.predict(channel)[0], axis = 2)
        #s_pred[s_pred!=1] = 0
        plt.imshow(s_pred)
        plt.savefig(f"/home/tordatom/Dati_Imaging/BraTs_19/Segmentation2D/Tracin_Images/new_totdice/opponents_of_test_{i}/train_opponent_{index}.png")
        plt.show()
        